<a href="https://colab.research.google.com/github/alesordo/LabAI/blob/main/LabAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Google Inception pre-trained model

In [ ]:
#@title Preliminar operations
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys
import random
from tqdm import tqdm
import shutil
import statistics
from os import mkdir
import math
import pathlib
from PIL import Image

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib
import csv

#Mounting directories
base_dir='/content/drive/MyDrive/Archive'
train_dir=base_dir+'/Train'
test_dir=base_dir+'/Test'

In [ ]:
#@title OLD - Operazioni preliminari
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Commonly used modules
import numpy as np
import os
import sys
import random
from tqdm import tqdm
import shutil
import statistics
from os import mkdir
import math
import pathlib
from PIL import Image

# Images, plots, display, and visualization
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import cv2
import IPython
from six.moves import urllib
import csv


#Fixing python random seed
random.seed(42)
#Fixing numpy random seed
np.random.seed(21)

#Mounting directories
base_dir='/content/drive/MyDrive/GTSRB'
train_dir=base_dir+'/Final_Training/Images'
test_dir=base_dir+'/Final_Test/Images'
validation_dir = base_dir+"/Final_Validation/Images"

In [ ]:
#@title EXECUTE ONLY ONCE - CREATE NEW CSV - NOT NEEDED I USED KAGGLE
#EXECUTE ONLY ONCE
#Merging annotation csv files and creating column with relative filepaths
n_classes = 43
all_annotation_filenames=[]
for n_class in tqdm(range(n_classes)) :
  n_class_string=str(n_class).zfill(5)
  single_annotation_filename=train_dir+'/'+n_class_string+'/'+'GT-'+n_class_string+'.csv'
  all_annotation_filenames.append(single_annotation_filename)
combined_annotation_array=[]
for f in all_annotation_filenames:
  single_annotation_df=pd.read_csv(f,sep=';')
  single_annotation_df['Filepath'] = single_annotation_df.apply(lambda row: str(row.ClassId).zfill(5)+'/'+row.Filename, axis=1)
  combined_annotation_array.append(single_annotation_df)
combined_annotation_csv=pd.concat(combined_annotation_array)
combined_annotation_csv.to_csv(train_dir+"/Annotations.csv", index=False, encoding='utf-8-sig')

In [ ]:
#@title Plotting distribution
#Plotting training images dimensions' distribution
annotation_file=train_dir+"/Annotations.csv"
data=pd.read_csv(annotation_file)
width = data['Width']
height = data['Height']
fig, ax = plt.subplots(1,2, figsize=(20,10))
fig.tight_layout()
ax[0].set(xlabel='Width of images')
ax[1].set(xlabel='Height of images')
sns.histplot(width, ax=ax[0], legend=False, bins=50, kde=True, color='forestgreen')
sns.histplot(height, ax=ax[1], legend=False, bins=50, kde=True, color='rebeccapurple')
print(statistics.mean(width))
print(statistics.mean(height))

In [ ]:
#@title OLD - dividing data in folders EXECUTE ONLY ONCE
#WARNING! EXECUTE THIS COMMAND ONLY ONCE - OLD
#Dividing data into training and validation (25% of the training dataset)

#Creating a validation directory
os.makedirs(validation_dir)

n_classes = 43

#Moving data
for n_class in tqdm(range(n_classes)) :
  n_class_string=str(n_class).zfill(5)
  annotation_file='GT-'+n_class_string+'.csv'
  dest_class_path = os.path.join(validation_dir, n_class_string)
  os.makedirs(dest_class_path)
  src_class_path = os.path.join(train_dir, n_class_string)
  list_files = os.listdir(src_class_path)
  if (annotation_file) in list_files:
    list_files.remove(annotation_file)
  rand_idx = random.sample(range(len(list_files)), math.ceil(len(list_files)/4))
  for idx in rand_idx :
    src_file = src_class_path + "/" + list_files[idx]
    shutil.move(src_file, dest_class_path)

In [ ]:
#@title IDK, to balance ds
#TO USE WHEN YOU'LL DEFINE DATAFRAMES, IN ORDER TO SHOW BALANCE/UNBALANCE OF THE DATASET
#df[train_dir].value_counts().plot(kind = 'bar')

In [ ]:
#Convert path strings into pathlib objects
data_path_train = pathlib.Path(train_dir)
data_path_validation = pathlib.Path(validation_dir)

In [ ]:
#@title OLD - Convert files from ppm to jpg EXECUTE ONLY ONCE
#WARNING! EXECUTE ONLY ONCE - OLD
#Function to convert files from .ppm to .jpg (.ppm are not accepted by Tensorflow)
#Getting paths of all the images as strings
all_image_paths_train = list(data_path_train.glob('*/*.ppm'))
for image_path in all_image_paths_train:
  directory_name = os.path.dirname(image_path)
  image_filename = '{}.jpg'.format(os.path.splitext(os.path.basename(image_path))[0])
  new_image_path = os.path.join(directory_name, image_filename)
  if(not os.path.exists(new_image_path)):
    image=Image.open(image_path)
    image.save(new_image_path)

all_image_paths_validation = list(data_path_validation.glob('*/*.ppm'))
for image_path in all_image_paths_validation:
  image=Image.open(image_path)
  image_filename = '{}.jpg'.format(os.path.splitext(os.path.basename(image_path))[0])
  directory_name = os.path.dirname(image_path)
  image.save(os.path.join(directory_name, image_filename))

In [ ]:
#@title Convert files from ppm to jpg - NOT NEEDED, I USED KAGGLE'S DATASET


In [ ]:
#@title OLD - to edit filepaths with jpg on csv EXECUTE ONLY ONCE
#WARNING! EXECUTE ONLY ONCE
#Editing relative filepaths from .ppm to .jpg format
annotation_file=train_dir+"/Annotations.csv"
data=pd.read_csv(annotation_file)
data['Filepath'] = data['Filepath'].str.replace('ppm','jpg')
data.to_csv(annotation_file)

In [ ]:
#@title Fixed variables
#Fixed variables
WIDTH=224
HEIGHT=224
N_CLASSES=43
BATCH_SIZE=30
N_CHANNELS = 3
CLASS_NAMES=list(range(N_CLASSES))

In [ ]:
#@title OLD - to get images paths
#OLD

#Fixed variables
WIDTH=224
HEIGHT=224
N_CLASSES=43
BATCH_SIZE=30
N_CHANNELS = 3
CLASS_NAMES=list(range(N_CLASSES))

#Getting paths of all the images as strings
all_image_paths_train = list(data_path_train.glob('*/*.jpg'))
all_image_paths_train = [str(path) for path in all_image_paths_train]

all_image_paths_validation = list(data_path_validation.glob('*/*.jpg'))
all_image_paths_validation = [str(path) for path in all_image_paths_validation]

#Counting number of images in each sets
image_count_train = len(all_image_paths_train)
image_count_validation = len(all_image_paths_validation)

In [ ]:
#@title OLD - to get labels from images path
#Getting class labels array for each set (through images paths)
label_names_train = sorted(int(item.name) for item in data_path_train.glob('*/') if item.is_dir())
label_names_validation = sorted(int(item.name) for item in data_path_validation.glob('*/') if item.is_dir())

label_to_index_train = dict((name, index) for index,name in enumerate(label_names_train))
label_to_index_val = dict((name, index) for index,name in enumerate(label_names_validation))

all_image_labels_train = [label_to_index_train[int(pathlib.Path(path).parent.name)] for path in all_image_paths_train]
all_image_labels_validation = [label_to_index_val[int(pathlib.Path(path).parent.name)] for path in all_image_paths_validation]

In [ ]:
#@title OLD - to update bounding box coordinates
#Updating bounding boxes coordinates on csv

#Loading dataframe
df_annotations = pd.read_csv(train_dir+"/Annotations.csv")

#Updating coordinates
for idx, row in tqdm(df_annotations.iterrows()):
  w = row['Width']
  h = row['Height']
  if w > WIDTH:
    diff = w - WIDTH
    df_annotations.iloc[idx, 5] = df_annotations.iloc[idx]['Roi.X2'] - diff
  else:
    diff = WIDTH - w
    df_annotations.iloc[idx, 5] = df_annotations.iloc[idx]['Roi.X2'] + diff
  if h > HEIGHT :
    diff = h - HEIGHT
    df_annotations.iloc[idx, 6] = df_annotations.iloc[idx]['Roi.Y2'] - diff
  else :
    diff = HEIGHT - h
    df_annotations.iloc[idx, 6] = df_annotations.iloc[idx]['Roi.Y2'] + diff

39209it [00:39, 999.52it/s]


In [ ]:
#@title OLD - to split dataframe with validation
#Splitting dataframes into training and validation
train_idx_list=[]
validation_idx_list=[]
print(all_image_paths_train[7763])
for path_train in tqdm(all_image_paths_train):
  train_idx_list.append(df_annotations[df_annotations['Filepath']==path_train[51:]].index[0])
for path_validation in tqdm(all_image_paths_validation):
  validation_idx_list.append(df_annotations[df_annotations['Filepath']==path_validation[53:]].index[0])

print(len(train_idx_list),len(validation_idx_list))

df_train = pd.DataFrame()
df_validation = pd.DataFrame()

df_train = df_train.append(df_annotations.iloc[train_idx_list], ignore_index = True)
df_validation = df_validation.append(df_annotations.iloc[validation_idx_list], ignore_index = True)

#Removing everything except bounding box coordinates from dataframes
df_train = df_train.drop(['Height', 'Width', 'ClassId', 'Filepath', 'Filename'], axis = 1)
df_validation = df_validation.drop(['Height', 'Width', 'ClassId', 'Filepath', 'Filename'], axis = 1)

In [ ]:
#@title OLD- to create data generators (important)
#Creating data generators
def tfdata_generator(images, labels, df, is_training, batch_size=BATCH_SIZE):
  #Construct a data generator using tf.Dataset
  def parse_function(filename, labels, df):
    #Function to preprocess the images
    #reading path 
    image_string = tf.io.read_file(filename)
    #decoding image
    image = tf.image.decode_jpeg(image_string, channels=N_CHANNELS)
    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    #Adjusting contrast and brightness of the image
    if tf.math.reduce_mean(image) < 0.3 :
      image = tf.image.adjust_contrast(image, 5)
      image = tf.image.adjust_brightness(image, 0.2)
    #resize the image
    image = tf.image.resize(image, [HEIGHT, WIDTH], method="nearest", preserve_aspect_ratio=False)
    # This will convert to float values in [0, 1]
    #image = tf.image.convert_image_dtype(image, tf.float32)
    #one hot coding for label
    #y = tf.one_hot(tf.cast(label, tf.uint8), N_CLASSES)
    #image = image/255.0
    return image, {"classification" : labels, "regression" : df}
    #return image
  
  ##creating a dataset from tensorslices
  dataset = tf.data.Dataset.from_tensor_slices((images, labels, df))
  if is_training:
    dataset = dataset.repeat(10).shuffle(30000)  # depends on sample size
    #dataset = dataset.repeat(10) #10 is the number of epochs
  else:
    dataset=dataset.repeat(10).shuffle(10000)
  # Transform and batch data at the same time
  dataset = dataset.map(parse_function)
  dataset = dataset.repeat(10)
  dataset = dataset.batch(batch_size)
  #prefetch the data into CPU/GPU
  dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
  return dataset

#Train and Validation data generators :
tf_image_generator_train = tfdata_generator(all_image_paths_train, all_image_labels_train, df_train, is_training=True, batch_size=BATCH_SIZE)
tf_image_generator_val = tfdata_generator(all_image_paths_validation, all_image_labels_validation, df_validation, is_training=False, batch_size=BATCH_SIZE)


In [ ]:
#@title OLD - to show images
for i, (image, label) in enumerate(tf_image_generator_train.take(12)):
    ax = plt.subplot(4, 4, i + 1)
    image1 = image.numpy().astype("float32")
    #plt.imshow(image.numpy().astype("uint8"))
    plt.imshow(np.squeeze(image1[0]))
    plt.axis("off")

In [ ]:
STEPS_PER_EPOCH = len(train_idx_list)//BATCH_SIZE
VALIDATION_STEPS = len(validation_idx_list)//BATCH_SIZE

In [ ]:
#VGC-16
from tensorflow.keras.applications.vgg16 import VGG16

base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

# Flatten the output layer to 1 dimension
x = layers.Flatten()(base_model.output)

# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)

# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
x = layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

vgghist = model.fit(tf_image_generator_train, validation_data = tf_image_generator_val, steps_per_epoch = 100, epochs = 10)

In [ ]:
%pip install efficientnet

In [ ]:
#@title OLD - Efficientnet
#EFFICIENTNET

import efficientnet.keras as efn
from tensorflow.keras import Model
from tensorflow.keras import optimizers


base_model = efn.EfficientNetB0(input_shape = (35, 35, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

# Add a final sigmoid layer with 1 node for classification output
predictions = Dense(1, activation="sigmoid")(x)
model_final = Model(inputs = base_model.input, outputs = predictions)

model_final.compile(optimizers.RMSprop(lr=0.0001, decay=1e-6),loss='binary_crossentropy',metrics=['accuracy'])
eff_history = model_final.fit_generator(tf_image_generator_train, validation_data = tf_image_generator_val, steps_per_epoch = 100, validation_steps=30, epochs = 10)
